In [3]:
#import relevant libraries
import pandas as pd
from matplotlib import pyplot as plt

In [13]:
#import data and check that data has been correctly imported
data = pd.read_csv("data/profiles.csv")
print(len(data))

59946


In [5]:
#set max_columns and max_colwidth to make it easier to see data in DataFrame
pd.options.display.max_columns = 60
pd.options.display.max_colwidth = 100

In [6]:
data.head()

age       body_type               diet    drinks      drugs  \
0   22  a little extra  strictly anything  socially      never   
1   35         average       mostly other     often  sometimes   
2   38            thin           anything  socially        NaN   
3   23            thin         vegetarian  socially        NaN   
4   29        athletic                NaN  socially      never   

                           education  \
0      working on college/university   
1              working on space camp   
2     graduated from masters program   
3      working on college/university   
4  graduated from college/university   

                                                                                                essay0  \
0  about me:<br />\n<br />\ni would love to think that i was some some kind of intellectual:\neithe...   
1  i am a chef: this is what that means.<br />\n1. i am a workaholic.<br />\n2. i love to cook rega...   
2  i'm not ashamed of much, but writing public text on an online\ndating site makes me pleasantly u...   
3                                                            i work in a library and go to school. . .   
4  hey how's it going? currently vague on the profile i know, more to\ncome soon. looking to meet n...   

                                                                                                essay1  \
0  currently working as an international agent for a freight\nforwarding company. import, export, d...   
1                                                 dedicating everyday to being an unbelievable badass.   
2  i make nerdy software for musicians, artists, and experimenters to\nindulge in their own weirdne...   
3                                                            reading things written by old dead people   
4                                                                           work work work work + play   

                                                                                                essay2  \
0  making people laugh.<br />\nranting about a good salting.<br />\nfinding simplicity in complexit...   
1    being silly. having ridiculous amonts of fun wherever. being a\nsmart ass. ohh and i can cook. ;)   
2  improvising in different contexts. alternating between being\npresent and decidedly outside of a...   
3  playing synthesizers and organizing books according to the library\nof congress classification s...   
4  creating imagery to look at:<br />\nhttp://bagsbrown.blogspot.com/<br />\nhttp://stayruly.blogsp...   

                                                                                                essay3  \
0  the way i look. i am a six foot half asian, half caucasian mutt. it\nmakes it tough not to notic...   
1                                                                                                  NaN   
2  my large jaw and large glasses are the physical things people\ncomment on the most. when suffici...   
3                                                                    socially awkward but i do my best   
4                                                              i smile a lot and my inquisitive nature   

                                                                                                essay4  \
0  books:<br />\nabsurdistan, the republic, of mice and men (only book that made me\nwant to cry), ...   
1  i am die hard christopher moore fan. i don't really watch a lot of\ntv unless there is humor inv...   
2  okay this is where the cultural matrix gets so specific, it's like\nbeing in the crosshairs.<br ...   
3  bataille, celine, beckett. . .<br />\nlynch, jarmusch, r.w. fassbender. . .<br />\ntwin peaks &a...   
4  music: bands, rappers, musicians<br />\nat the moment: thee oh sees.<br />\nforever: wu-tang<br ...   

                                                                                                essay5  \
0                                               food.<br />\nwater.<br />\

In [9]:
#explore range of values in diet column
data['diet'].value_counts()

mostly anything        16585
anything                6183
strictly anything       5113
mostly vegetarian       3444
mostly other            1007
strictly vegetarian      875
vegetarian               667
strictly other           452
mostly vegan             338
other                    331
strictly vegan           228
vegan                    136
mostly kosher             86
mostly halal              48
strictly halal            18
strictly kosher           18
kosher                    11
halal                     11
Name: diet, dtype: int64

In [8]:
#explore range of values in income column
data['income'].value_counts()

-1          48442
 20000       2952
 100000      1621
 80000       1111
 30000       1048
 40000       1005
 50000        975
 60000        736
 70000        707
 150000       631
 1000000      521
 250000       149
 500000        48
Name: income, dtype: int64

In [14]:
#drop features from DataFrame and check if remaining features have NaN values - if so, remove them
features_to_remove = [
    'body_type', 
    'education', 
    'ethnicity', 
    'height', 
    'job', 
    'last_online', 
    'location', 
    'offspring',
    'orientation',
    'pets',
    'sex', 
    'sign',
    'speaks',
    'status',
    'religion'
]
data.drop(features_to_remove, axis=1, inplace=True)
data.isna().any()

age       False
diet       True
drinks     True
drugs      True
essay0     True
essay1     True
essay2     True
essay3     True
essay4     True
essay5     True
essay6     True
essay7     True
essay8     True
essay9     True
income    False
smokes     True
dtype: bool

In [27]:
#create DataFrame with just diet and income columns from original DataFrame
income_diet_age_df = data[['diet', 'income', 'age']].copy()
print(len(income_diet_age_df))
income_diet_df.head()

59946


diet  income  age
0  strictly anything      -1   22
1       mostly other   80000   35
2           anything      -1   38
3         vegetarian   20000   23
4                NaN      -1   29

In [28]:
#check if either diet or income column have NaN values and drop
income_diet_age_df.isna().any()

diet       True
income    False
age       False
dtype: bool

In [34]:
#drop NaN values in diet column
income_diet_age_df_no_nan = income_diet_age_df.dropna(subset=['diet'])
print(len(income_diet_df_no_nan))
income_diet_age_df_no_nan.isna().any()

35551


diet      False
income    False
age       False
dtype: bool

In [31]:
income_diet_age_df_no_nan['income'].value_counts()

-1          28086
 20000       2003
 100000      1047
 80000        712
 30000        661
 40000        647
 50000        592
 70000        446
 60000        442
 150000       407
 1000000      377
 250000        96
 500000        35
Name: income, dtype: int64

In [35]:
#drop rows where income value is -1
income_diet_age_df_cleaned = income_diet_age_df_no_nan[income_diet_age_df_no_nan.income != -1]
print(len(income_diet_age_df_cleaned))

7465


In [38]:
#map different diets to three diet codes - 1 for anything vegan, 2 for anything vegetarian, and 3 for all other diet types
def diet_mappings(diet):
    if "vegan" in diet:
        return 1
    elif "vegetarian" in diet:
        return 2
    else:
        return 3

income_diet_age_df_cleaned['diet_code'] = income_diet_age_df_cleaned['diet'].apply(diet_mappings)
income_diet_age_df_cleaned['diet_code'].value_counts()

/Users/Andrew/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


3    6384
2     911
1     170
Name: diet_code, dtype: int64

In [46]:
#Based on above error - and research on StackOverflow - have opted to create a copy of the DataFrame and assign diet_mappings to copy
analysis_df = income_diet_age_df_cleaned.copy()
analysis_df['diet_code'] = income_diet_age_df_cleaned['diet'].apply(diet_mappings)

7465
